## Purpose: Try different models-- Part2.
### Logistic Regression with scaling.

In [1]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../Resources/clean_data.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,St. Louis Cardinals,2019,1033,114,43,936,8313.0,3,2771,3847,...,0.319,456,4,895,33,3896,56,1.29,21,0
1,Arizona Diamondbacks,2019,1010,83,45,945,8538.0,2,2846,3901,...,0.315,472,7,925,24,4001,53,1.28,35,0
2,Kansas City Royals,2019,990,105,45,954,8421.0,6,2807,3842,...,0.346,543,5,816,24,4125,39,1.46,34,0
3,Houston Astros,2019,875,54,50,954,8589.0,6,2863,3788,...,0.284,432,7,1074,27,3929,67,1.14,31,0
4,Tampa Bay Rays,2019,975,92,53,963,8760.0,11,2920,3948,...,0.291,409,6,1037,26,3985,59,1.16,40,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
120,San Francisco Giants,2015,1639,136,72,1458,13143.0,6,4381,6092,...,0.303,631,11,1309,43,6048,87,1.21,40,0
121,Washington Nationals,2015,1425,142,73,1458,13137.0,17,4379,5877,...,0.300,612,12,1476,46,6036,95,1.19,47,0
122,Houston Astros,2015,1599,135,77,1458,13212.0,18,4404,6080,...,0.314,701,8,1396,44,6180,84,1.29,98,0
123,Detroit Tigers,2015,1537,148,75,1449,12852.0,5,4284,5896,...,0.320,721,8,1232,47,6048,86,1.32,44,0
124,Boston Red Sox,2015,1427,139,75,1458,12957.0,37,4319,5821,...,0.314,694,5,1362,43,6073,93,1.27,52,0


In [4]:
# models require numbers! Drop team and year columns.
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
print (target.shape)
print (features.shape)

(2344,)
(2344, 46)


In [5]:
# split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

#### STEP2: Try Logistic Regression.

In [6]:
# reshape the target data.
target1 = target.values.reshape(-1, 1)

In [7]:
# split data into train and test sets.
X_train1, X_test1, y_train1, y_test1 = train_test_split(features, target1, random_state=42)

In [8]:
# create standard scaler model and fit the training data.
X_scaler = StandardScaler().fit(X_train1)
y_scaler = StandardScaler().fit(y_train1)

# transform the training and testing data.
X_train_scaled = X_scaler.transform(X_train1)
X_test_scaled = X_scaler.transform(X_test1)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [9]:
# fit the model.
model = LogisticRegression(solver="lbfgs")
model.fit(X_train_scaled, y_train1)

# predict.
prediction = model.predict(X_test_scaled)

# evaluate the model.
mse = mean_squared_error(y_test1, prediction)
r2 = model.score(X_test_scaled, y_test1)
print (f"MSE: {mse}")
print (f"R2: {r2}")

MSE: 0.05631399317406143
R2: 0.9436860068259386


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wow.  That's a pretty good fit!  Since the scores are so high, try bs test with 2016-2018 data. But wait...  

In [10]:
print (classification_report(y_test1, prediction, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       553
           1       0.50      0.03      0.06        33

   micro avg       0.94      0.94      0.94       586
   macro avg       0.72      0.51      0.51       586
weighted avg       0.92      0.94      0.92       586



Ah hah!.  The model is predicting 0's correctly but only predicts 1's 50% of the time.  And it isn't terribly accurate.  That means that the two classes (0, 1) are very imbalanced.  Need to address this one.

#### STEP3: Try Logistic Regression with Upsampled Data.

In [11]:
# start over.  grab the data.
team_data_new = team_data_new.reset_index().drop({"index"}, axis=1)
team_data_new.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,San Francisco Giants,2015,1639,136,72,1458,13143.0,6,4381,6092,...,0.303,631,11,1309,43,6048,87,1.21,40,0
1,Washington Nationals,2015,1425,142,73,1458,13137.0,17,4379,5877,...,0.300,612,12,1476,46,6036,95,1.19,47,0
2,Houston Astros,2015,1599,135,77,1458,13212.0,18,4404,6080,...,0.314,701,8,1396,44,6180,84,1.29,98,0
3,Detroit Tigers,2015,1537,148,75,1449,12852.0,5,4284,5896,...,0.320,721,8,1232,47,6048,86,1.32,44,0
4,Boston Red Sox,2015,1427,139,75,1458,12957.0,37,4319,5821,...,0.314,694,5,1362,43,6073,93,1.27,52,0


In [12]:
# remove team and year.
team_data_new = team_data_new.drop({"team", "year"}, axis=1)

In [13]:
# upsample for a more balanced dataset.
def upsample(dataset, no_samples):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    -n_samples = number of minority_unsampled.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -dataset is taken in and split into minority and majority classes.
    -dataset is then upsampled for the mainority class
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate majority and minority classes.
    df_majority = dataset.loc[dataset["winners"] == 0]
    df_minority = dataset.loc[dataset["winners"] == 1]

    # upsample minority class.
    df_minority_unsampled = resample(df_minority,
                                    replace=True,
                                    n_samples=no_samples,
                                    random_state=123)

    # combine majority class with upsampled minority class.
    df_upsampled = pd.concat([df_majority, df_minority_unsampled])

    # separate features and target.
    y = df_upsampled["winners"]
    X = df_upsampled.drop("winners", axis=1)
    
    # split into train and test sets.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # scale X_train and X_test.
    scaler = StandardScaler()

    # transform the training and testing data.
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [14]:
# Do three different upsamplings.
X_train_100, X_test_100, y_train_100, y_test_100 = upsample(team_data_new, 2234)
X_train_50, X_test_50, y_train_50, y_test_50 = upsample(team_data_new, 1117)
X_train_25, X_test_25, y_train_25, y_test_25 = upsample(team_data_new, 559)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anacon

In [15]:
def fit_logistic(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -model = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a logistic regression.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # fit the model.
    model = LogisticRegression(solver="lbfgs", max_iter= 2000)
    model.fit(X_train, y_train)

    # predict.
    prediction = model.predict(X_test)
    
    # print out the classification report
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return model

In [16]:
# for 1 part 0 to 1 part 1
model_100 = fit_logistic(X_train_100, X_test_100, y_train_100, y_test_100)

              precision    recall  f1-score   support

           0       0.74      0.69      0.72       573
           1       0.70      0.75      0.72       544

   micro avg       0.72      0.72      0.72      1117
   macro avg       0.72      0.72      0.72      1117
weighted avg       0.72      0.72      0.72      1117



In [17]:
# for 1 part 0 to 0.5 part 1
model_50 = fit_logistic(X_train_50, X_test_50, y_train_50, y_test_50)

              precision    recall  f1-score   support

           0       0.81      0.86      0.84       570
           1       0.67      0.58      0.62       268

   micro avg       0.77      0.77      0.77       838
   macro avg       0.74      0.72      0.73       838
weighted avg       0.77      0.77      0.77       838



In [18]:
# for 1 part 0 to 0.25 part 1
model_25 = fit_logistic(X_train_25, X_test_25, y_train_25, y_test_25)

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       556
           1       0.70      0.35      0.47       143

   micro avg       0.84      0.84      0.84       699
   macro avg       0.78      0.66      0.69       699
weighted avg       0.82      0.84      0.81       699



Ok.  That looks a little better -- at least it is predicting 0 and 1s.  The model_100 seems to be most even between predicting 0 and 1s.  Use that one.

#### STEP4: Predict 2016-2018 winners.

In [39]:
def predict_the_winner(model, year, team_data, X_train):
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()

    # set features (no team, year, winners).
    # set target (winners).
    features = team_data.drop({"team", "year", "winners"}, axis=1).drop({"index"}, axis=1)
    # scale X_train and X_test.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    features = scaler.fit_transform(features)
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i,1:27]["team"])

##### 2018 Prediction

In [40]:
# predict for 2018.
predict_the_winner(model_100, 2018, team_data, X_train_100)


WS Probability = 0.9973075059180571
Los Angeles Dodgers

WS Probability = 0.9968369191970068
Houston Astros

WS Probability = 0.9949572033336812
Cleveland Indians

WS Probability = 0.9948879011452587
Minnesota Twins

WS Probability = 0.9933862777762907
Cincinnati Reds

WS Probability = 0.992237333810715
Washington Nationals

WS Probability = 0.9838994258451377
St. Louis Cardinals

WS Probability = 0.941337290976353
Arizona Diamondbacks

WS Probability = 0.900302629483727
Oakland Athletics

WS Probability = 0.8005904185104195
Tampa Bay Rays

WS Probability = 0.7482051044015635
Boston Red Sox

WS Probability = 0.7191205218502633
Atlanta Braves

WS Probability = 0.6984506022000315
New York Yankees

WS Probability = 0.6593690633608191
Los Angeles Angels

WS Probability = 0.4298264598691424
Miami Marlins

WS Probability = 0.256736751457773
New York Mets

WS Probability = 0.1881393535506471
Chicago White Sox

WS Probability = 0.17367347558480653
Texas Rangers

WS Probability = 0.11528540543

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


This does not look good.  The Red Sox won 2018.

##### 2017 Prediction

In [41]:
# predict for 2017.
predict_the_winner(model_100, 2017, team_data, X_train_100)


WS Probability = 0.9999812992208802
Houston Astros

WS Probability = 0.9999601596560308
Cleveland Indians

WS Probability = 0.9969776314222628
Boston Red Sox

WS Probability = 0.9958674146086319
Atlanta Braves

WS Probability = 0.9860615391272469
Washington Nationals

WS Probability = 0.9837480005463379
Oakland Athletics

WS Probability = 0.9757363214848371
Los Angeles Angels

WS Probability = 0.9691692321808807
New York Yankees

WS Probability = 0.9566651596158129
Los Angeles Dodgers

WS Probability = 0.947990474669474
Tampa Bay Rays

WS Probability = 0.8861121162511377
Seattle Mariners

WS Probability = 0.5423618672667077
Pittsburgh Pirates

WS Probability = 0.5347244929536942
New York Mets

WS Probability = 0.387685598639102
Detroit Tigers

WS Probability = 0.34719797742971537
Minnesota Twins

WS Probability = 0.2830331058190067
Colorado Rockies

WS Probability = 0.24314952529186085
Milwaukee Brewers

WS Probability = 0.16956036771945562
Kansas City Royals

WS Probability = 0.16295

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


And the Astros won 2017.  Huh.  This one worked.

##### 2016 Prediction

In [42]:
# predict for 2016.
predict_the_winner(model_100, 2016, team_data, X_train_100)


WS Probability = 0.9999994527851971
Cleveland Indians

WS Probability = 0.9984233317506738
Houston Astros

WS Probability = 0.9969613026882935
Los Angeles Dodgers

WS Probability = 0.9947686622578362
New York Yankees

WS Probability = 0.9934572515631251
Washington Nationals

WS Probability = 0.9738251162429734
Minnesota Twins

WS Probability = 0.9630591696769951
Boston Red Sox

WS Probability = 0.9248134254353411
Kansas City Royals

WS Probability = 0.7708704132879565
Arizona Diamondbacks

WS Probability = 0.7416238673150909
Los Angeles Angels

WS Probability = 0.7065784008253861
Seattle Mariners

WS Probability = 0.6361148939905463
St. Louis Cardinals

WS Probability = 0.5531813000503839
Detroit Tigers

WS Probability = 0.3395340985718916
Chicago Cubs

WS Probability = 0.2639533650083351
Tampa Bay Rays

WS Probability = 0.25995898569572856
Colorado Rockies

WS Probability = 0.19246969606263253
New York Mets

WS Probability = 0.1712862326697155
Oakland Athletics

WS Probability = 0.09

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


The Cubs won 2016.  Two of three predictions were incorrect.  I want to try another model--want to improve F1 score.